Recommend to a user the exact item they would prefer is a win. However, user situations and tastes change frequently with flooding offers from everywhere.

Session-based sequential recommendation systems help to capture short-term, contextual user preferences at a given point in time and predict more precisely the next iterations.

NVIDIA-Merlin introduced #Transformers4Rec library to tackle session-based recommendation tasks by leveraging SOTA NLP Hugging Face #transformers architecture.

Simply use a short sequence of past user interactions within a session to recommend dynamically the next item they most likely would be interested in at the exact moment!
PyTorch and Tensorflow implementations are kindly available :)

In [ ]:
"""
All: pip install transformers4rec[all]
PyTorch: pip install transformers4rec[pytorch,nvtabular]
Tensorflow: pip install transformers4rec[tensorflow,nvtabular]

"""

## PyTorch implementation

In [ ]:
!pip install transformers4rec[pytorch,nvtabular]

In [12]:
from transformers4rec import torch as tr

In [22]:
schema: tr.Schema = tr.data.tabular_sequence_testing_data.schema
# Or read schema from disk: tr.Schema().from_json(SCHEMA_PATH)
max_sequence_length, d_model = 20, 64

In [23]:
print(f"Schema structure:\n{schema}")

Schema structure:
[{'name': 'session_id', 'type': 'INT', 'int_domain': {'name': 'session_id', 'min': '1', 'max': '11562158'}, 'annotation': {'tag': ['groupby_col']}}, {'name': 'session_start', 'annotation': {'tag': ['first']}}, {'name': 'timestamp/age_days/list', 'value_count': {'min': '2', 'max': '185'}, 'type': 'INT', 'int_domain': {'name': 'timestamp/age_days/list', 'max': '181'}, 'annotation': {'tag': ['list']}}, {'name': 'timestamp/age_days/LogOp/Normalize/list', 'value_count': {'min': '2', 'max': '185'}, 'type': 'FLOAT', 'float_domain': {'name': 'timestamp/age_days/LogOp/Normalize/list', 'min': -2.917729139328003, 'max': 1.5231701135635376}, 'annotation': {'tag': ['continuous', 'list']}}, {'name': 'timestamp/hour/list', 'value_count': {'min': '2', 'max': '185'}, 'type': 'FLOAT', 'float_domain': {'name': 'timestamp/hour/list', 'min': 5.7866054703481495e-06, 'max': 1.605135440826416}, 'annotation': {'tag': ['continuous', 'time', 'list']}}, {'name': 'timestamp/weekday/list', 'value_

In [24]:
import pandas as pd
pd.DataFrame(schema).head(3)

,name,deprecated,presence,group_presence,shape,value_count,value_counts,type,domain,int_domain,...,time_domain,time_of_day_domain,distribution_constraints,annotation,skew_comparator,drift_comparator,in_environment,not_in_environment,lifecycle_stage,unique_constraints
0,session_id,False,"{'min_fraction': 0.0, 'min_count': 0}",{'required': False},{'dim': []},"{'min': 0, 'max': 0}",{'value_count': []},2,,"{'name': 'session_id', 'min': 1, 'max': 115621...",...,"{'string_format': '', 'integer_format': 0}","{'string_format': '', 'integer_format': 0}",{'min_domain_mass': 0.0},"{'tag': ['groupby_col'], 'comment': [], 'extra...","{'infinity_norm': {'threshold': 0.0}, 'jensen_...","{'infinity_norm': {'threshold': 0.0}, 'jensen_...",[],[],0,"{'min': 0, 'max': 0}"
1,session_start,False,"{'min_fraction': 0.0, 'min_count': 0}",{'required': False},{'dim': []},"{'min': 0, 'max': 0}",{'value_count': []},0,,"{'name': '', 'min': 0, 'max': 0, 'is_categoric...",...,"{'string_format': '', 'integer_format': 0}","{'string_format': '', 'integer_format': 0}",{'min_domain_mass': 0.0},"{'tag': ['first'], 'comment': [], 'extra_metad...","{'infinity_norm': {'threshold': 0.0}, 'jensen_...","{'infinity_norm': {'threshold': 0.0}, 'jensen_...",[],[],0,"{'min': 0, 'max': 0}"
2,timestamp/age_days/list,False,"{'min_fraction': 0.0, 'min_count': 0}",{'required': False},{'dim': []},"{'min': 2, 'max': 185}",{'value_count': []},2,,"{'name': 'timestamp/age_days/list', 'min': 0, ...",...,"{'string_format': '', 'integer_format': 0}","{'string_format': '', 'integer_format': 0}",{'min_domain_mass': 0.0},"{'tag': ['list'], 'comment': [], 'extra_metada...","{'infinity_norm': {'threshold': 0.0}, 'jensen_...","{'infinity_norm': {'threshold': 0.0}, 'jensen_...",[],[],0,"{'min': 0, 'max': 0}"


Define input module to process tabular input-features

In [25]:
input_module = tr.TabularSequenceFeatures.from_schema(
    schema,
    max_sequence_length=max_sequence_length,
    continuous_projection=d_model,
    aggregation="concat",
    masking="causal",
)

Define one or multiple prediction-tasks

In [26]:
prediction_tasks = tr.NextItemPredictionTask()

Define a transformer-config, like the XLNet architecture

In [27]:
transformer_config = tr.XLNetConfig.build(
    d_model=d_model, n_head=4, n_layer=2, total_seq_length=max_sequence_length
)
model: tr.Model = transformer_config.to_torch_model(input_module, prediction_tasks)

## Tensorflow implementation

In [ ]:
!pip install transformers4rec[tensorflow,nvtabular]

In [16]:
!python -V

Python 3.7.13


In [20]:
from transformers4rec import tf as tr
from merlin_standard_lib import Schema

In [29]:
# schema = Schema().from_proto_text("<schema path>")
max_sequence_length, d_model = 20, 320

Define input module to process tabular input-features and to prepare masked inputs

In [ ]:
input_module = tr.TabularSequenceFeatures.from_schema(
    schema,
    max_sequence_length=max_sequence_length,
    continuous_projection=64,
    aggregation="concat",
    d_output=d_model,
    masking="clm",
)

Define Next item prediction-task 

In [ ]:
prediction_task = tr.NextItemPredictionTask(hf_format=True,weight_tying=True)

Define the config of the XLNet architecture

In [ ]:
transformer_config = tr.XLNetConfig.build(
    d_model=d_model, n_head=8, n_layer=2,total_seq_length=max_sequence_length
)

Get the TF model 

In [ ]:
model = transformer_config.to_tf_model(input_module, prediction_task)